<a href="https://colab.research.google.com/github/crneubert/best-music/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SETUP**


In [7]:
!pip install pyspark
!pip install -U -q PyDrive

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"] += ":/usr/lib/jvm/java-17-openjdk-amd64/bin"

In [9]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import types as sparktypes

# import PySpark aggregate functions with underscores to avoid collision with Python sum, etc
from pyspark.sql.functions import sum as _sum, avg as _avg, count as _count
from pyspark.sql.functions import col, lit, round, month, to_date, when, expr

from pyspark.sql.window import Window
from pyspark.sql.functions import rank, dense_rank, row_number, lag, lead

In [8]:
!wget -q https://media.githubusercontent.com/media/crneubert/best-music/refs/heads/main/data/calendar.csv
!wget -q https://media.githubusercontent.com/media/crneubert/best-music/refs/heads/main/data/listings.csv
!wget -q https://media.githubusercontent.com/media/crneubert/best-music/refs/heads/main/data/reviews.csv

In [10]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)

/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [11]:
listings = sqlContext.read.csv("listings.csv", header = True)
reviews = sqlContext.read.csv("reviews.csv", header = True)
calendar = sqlContext.read.csv("calendar.csv", header = True)

# **ANALYSIS**

In [12]:
listings.show()

+-----+--------------------+---------+---------+--------------------+--------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|   id|                name|  host_id|host_name| neighbourhood_group| neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+-----+--------------------+---------+---------+--------------------+--------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
| 6369|Rooftop terrace r...|    13660|    Simon|           Chamartín|Hispanoamérica|40.45724| -3.67688|   Private room|   60|             1|               78| 2020-09-20|             0.58|                             1|             180|
|21853|Bright and airy room|    83531|    Abdel|    

In [13]:
reviews.show()

+----------+----------+
|listing_id|      date|
+----------+----------+
|      6369|2010-03-14|
|      6369|2010-03-23|
|      6369|2010-04-10|
|      6369|2010-04-21|
|      6369|2010-04-26|
|      6369|2010-05-10|
|      6369|2010-05-15|
|      6369|2010-05-23|
|      6369|2010-05-24|
|      6369|2010-06-25|
|      6369|2010-06-28|
|      6369|2010-09-09|
|      6369|2010-09-12|
|      6369|2010-09-15|
|      6369|2010-09-27|
|      6369|2010-10-10|
|      6369|2010-10-14|
|      6369|2010-11-02|
|      6369|2010-11-14|
|      6369|2010-11-28|
+----------+----------+
only showing top 20 rows



In [14]:
calendar.show()

+----------+----------+---------+------+--------------+--------------+--------------+
|listing_id|      date|available| price|adjusted_price|minimum_nights|maximum_nights|
+----------+----------+---------+------+--------------+--------------+--------------+
|    167183|2021-04-15|        f|$45.00|        $45.00|             1|             5|
|      6369|2021-04-15|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-16|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-17|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-18|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-19|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-20|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-21|        t|$60.00|        $60.00|             1|          1125|
|      6369|2021-04-22|        t|$60.00|        $60.00

In [36]:
listings_clean = (listings.select("id", "room_type", "price", "minimum_nights", "number_of_reviews")
                         .withColumnRenamed("id", "listing_id")
                         .filter(col("room_type").isin("Shared room", "Private room", "Entire home/apt", "Hotel room")))



calendar_clean = (calendar.withColumn("available_boolean", when(col("available") == "t", 0).otherwise(1))
                          .groupBy("listing_id")
                          .agg(_avg("available_boolean").alias("occupancy_rate")))

combo_pizza = listings_clean.join(calendar_clean, on="listing_id")




In [37]:
room_type = (combo_pizza.groupBy("room_type")
                        .agg(_avg("occupancy_rate").alias("avg_occupancy_rate"))
                        .orderBy("avg_occupancy_rate"))
room_type.show()

+---------------+------------------+
|      room_type|avg_occupancy_rate|
+---------------+------------------+
|     Hotel room|0.3468134414697336|
|    Shared room|0.5122921401780076|
|Entire home/apt| 0.529229739202593|
|   Private room| 0.555126992439271|
+---------------+------------------+



In [33]:
listings.groupBy("room_type").count().collect()

[Row(room_type='15', count=2),
 Row(room_type='42', count=1),
 Row(room_type='Shared room', count=328),
 Row(room_type='59', count=1),
 Row(room_type='-3.70739', count=1),
 Row(room_type='22', count=3),
 Row(room_type='35', count=2),
 Row(room_type='16', count=2),
 Row(room_type='410', count=1),
 Row(room_type=None, count=58),
 Row(room_type='43', count=1),
 Row(room_type='100', count=1),
 Row(room_type='18', count=1),
 Row(room_type='61', count=1),
 Row(room_type='Hotel room', count=166),
 Row(room_type='40.42756', count=1),
 Row(room_type='78', count=1),
 Row(room_type='1200', count=1),
 Row(room_type='90', count=2),
 Row(room_type='19', count=1),
 Row(room_type='23', count=1),
 Row(room_type='55', count=1),
 Row(room_type='Entire home/apt', count=11286),
 Row(room_type='-3.67316', count=1),
 Row(room_type='38', count=1),
 Row(room_type='40', count=2),
 Row(room_type='376', count=1),
 Row(room_type='40.40902', count=1),
 Row(room_type='115', count=1),
 Row(room_type='-3.70747', count